# Part 6. 데이터프레임의 다양한 응용

## 1. 함수 매핑

함수 매핑: 시리즈 또는 데이터프레임의 개별 원소를 특정 함수에 일대일 대응시키는 과정

### 1-1. 개별 원소에 함수 매핑  

* 시리즈 원소에 함수 매핑  

apply() 메소드 적용  
: `Series객체.apply(매핑 함수)`

In [1]:
import seaborn as sns

titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age','fare']]
df['ten'] = 10
print(df.head())

    age     fare  ten
0  22.0   7.2500   10
1  38.0  71.2833   10
2  26.0   7.9250   10
3  35.0  53.1000   10
4  35.0   8.0500   10


In [2]:
# 사용자 함수 정의
def add_10(n):
    return n + 10

def add_two_obj(a, b):
    return a + b

print(add_10(10))
print(add_two_obj(10, 10))

20
20


In [3]:
sr1 = df['age'].apply(add_10)
print(sr1.head())
print('\n')

sr2 = df['age'].apply(add_two_obj, b=10)
print(sr2.head())
print('\n')

# lambda 함수 활용
sr3 = df['age'].apply(lambda x: add_10(x))
print(sr3.head())

0    32.0
1    48.0
2    36.0
3    45.0
4    45.0
Name: age, dtype: float64


0    32.0
1    48.0
2    36.0
3    45.0
4    45.0
Name: age, dtype: float64


0    32.0
1    48.0
2    36.0
3    45.0
4    45.0
Name: age, dtype: float64


* 데이터프레임 원소에 함수 매핑  

applymap() 메소드 적용  
: `DataFrame객체.applymap(매핑 함수)`

In [4]:
df_map = df.applymap(add_10)
print(df_map.head())

    age     fare  ten
0  32.0  17.2500   20
1  48.0  81.2833   20
2  36.0  17.9250   20
3  45.0  63.1000   20
4  45.0  18.0500   20


### 1-2. 시리즈 객체에 함수 매핑

* 데이터프레임의 각 열에 함수 매핑  

: `DataFrame객체.apply(매핑 함수, axis=0)`

In [5]:
import seaborn as sns

titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age','fare']]

In [6]:
# 사용자 함수 정의
def missing_value(series):
    return series.isnull() # 불린 시리즈를 반환

In [7]:
result = df.apply(missing_value, axis=0)
print(result.head())
print('\n')
print(type(result))

     age   fare
0  False  False
1  False  False
2  False  False
3  False  False
4  False  False


<class 'pandas.core.frame.DataFrame'>


In [8]:
def min_max(x):
    return x.max() - x.min() # 하나의 값을 반환

In [9]:
result = df.apply(min_max) # 기본값 axis=0
print(result)
print('\n')
print(type(result))

age      79.5800
fare    512.3292
dtype: float64


<class 'pandas.core.series.Series'>


* 데이터프레임의 각 행에 함수 매핑  

: `DataFrame객체.apply(매핑 함수, axis=1)`

In [10]:
import seaborn as sns

titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age','fare']]
df['ten'] = 10

def add_two_obj(a, b):
    return a + b

In [11]:
df['add'] = df.apply(lambda x: add_two_obj(x['age'], x['ten']), axis=1)
print(df.head())

    age     fare  ten   add
0  22.0   7.2500   10  32.0
1  38.0  71.2833   10  48.0
2  26.0   7.9250   10  36.0
3  35.0  53.1000   10  45.0
4  35.0   8.0500   10  45.0


### 1-3. 데이터프레임 객체에 함수 매핑  

pipe() 메소드 적용  
사용하는 함수가 반환하는 리턴값에 따라 pipe() 메소드가 반환하는 객체의 종류가 결정된다.(데이터프레임, 시리즈, 값)  
: `DataFrame객체.pipe(매핑 함수)`

In [12]:
import seaborn as sns

titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age','fare']]

# 데이터프레임 -> 데이터프레임
def missing_value(x):
    return x.isnull()

# 데이터프레임 -> 시리즈
def missing_count(x):
    return missing_value(x).sum()

# 데이터프레임 -> 값
def total_number_missing(x):
    return missing_count(x).sum()

In [13]:
result_df = df.pipe(missing_value)
print(result_df.head())
print('\n')
print(type(result_df))

     age   fare
0  False  False
1  False  False
2  False  False
3  False  False
4  False  False


<class 'pandas.core.frame.DataFrame'>


In [14]:
result_series = df.pipe(missing_count)
print(result_series)
print('\n')
print(type(result_series))

age     177
fare      0
dtype: int64


<class 'pandas.core.series.Series'>


In [15]:
result_value = df.pipe(total_number_missing)
print(result_value)
print('\n')
print(type(result_value))

177


<class 'numpy.int64'>


## 2. 열 재구성

### 2-1. 열 순서 변경  

: `DataFrame객체[재구성한 열 이름의 리스트]`

In [16]:
import seaborn as sns

titanic = sns.load_dataset('titanic')
df = titanic.loc[0:4, 'survived':'age']
print(df)

   survived  pclass     sex   age
0         0       3    male  22.0
1         1       1  female  38.0
2         1       3  female  26.0
3         1       1  female  35.0
4         0       3    male  35.0


In [17]:
columns = list(df.columns.values) # 기존 열 이름
print(columns)

['survived', 'pclass', 'sex', 'age']


In [18]:
columns_sorted = sorted(columns) # 알파벳 순으로 정렬
df_sorted = df[columns_sorted]
print(df_sorted)

    age  pclass     sex  survived
0  22.0       3    male         0
1  38.0       1  female         1
2  26.0       3  female         1
3  35.0       1  female         1
4  35.0       3    male         0


In [19]:
columns_reversed = list(reversed(columns)) # 기존 순서의 역순으로 정렬
df_reversed = df[columns_reversed]
print(df_reversed)

    age     sex  pclass  survived
0  22.0    male       3         0
1  38.0  female       1         1
2  26.0  female       3         1
3  35.0  female       1         1
4  35.0    male       3         0


In [20]:
columns_customed = ['pclass', 'sex', 'age', 'survived']
df_customed = df[columns_customed]
print(df_customed)

   pclass     sex   age  survived
0       3    male  22.0         0
1       1  female  38.0         1
2       3  female  26.0         1
3       1  female  35.0         1
4       3    male  35.0         0


### 2-2. 열 분리

In [21]:
import pandas as pd

df = pd.read_excel('주가데이터.xlsx')
print(df.head(), '\n')
print(df.dtypes)

         연월일   당일종가  전일종가     시가     고가     저가     거래량
0 2018-07-02  10100   600  10850  10900  10000  137977
1 2018-06-29  10700   300  10550  10900   9990  170253
2 2018-06-28  10400   500  10900  10950  10150  155769
3 2018-06-27  10900   100  10800  11050  10500  133548
4 2018-06-26  10800   350  10900  11000  10700   63039 

연월일     datetime64[ns]
당일종가             int64
전일종가             int64
시가               int64
고가               int64
저가               int64
거래량              int64
dtype: object


In [22]:
df['연월일'] = df['연월일'].astype('str')
dates = df['연월일'].str.split('-')
print(dates.head())

0    [2018, 07, 02]
1    [2018, 06, 29]
2    [2018, 06, 28]
3    [2018, 06, 27]
4    [2018, 06, 26]
Name: 연월일, dtype: object


In [23]:
df['연'] = dates.str.get(0)
df['월'] = dates.str.get(1)
df['일'] = dates.str.get(2)
print(df.head())

          연월일   당일종가  전일종가     시가     고가     저가     거래량     연   월   일
0  2018-07-02  10100   600  10850  10900  10000  137977  2018  07  02
1  2018-06-29  10700   300  10550  10900   9990  170253  2018  06  29
2  2018-06-28  10400   500  10900  10950  10150  155769  2018  06  28
3  2018-06-27  10900   100  10800  11050  10500  133548  2018  06  27
4  2018-06-26  10800   350  10900  11000  10700   63039  2018  06  26


## 3. 필터링

### 3-1. 불린 인덱싱

데이터프레임의 불린 인덱싱: `DataFrame객체[불린 시리즈]`

In [24]:
import seaborn as sns

mask1 = (titanic.age >= 10) & (titanic.age < 20)
df_teenage = titanic.loc[mask1, :]
print(df_teenage.head())

    survived  pclass     sex   age  sibsp  parch      fare embarked   class  \
9          1       2  female  14.0      1      0   30.0708        C  Second   
14         0       3  female  14.0      0      0    7.8542        S   Third   
22         1       3  female  15.0      0      0    8.0292        Q   Third   
27         0       1    male  19.0      3      2  263.0000        S   First   
38         0       3  female  18.0      2      0   18.0000        S   Third   

      who  adult_male deck  embark_town alive  alone  
9   child       False  NaN    Cherbourg   yes  False  
14  child       False  NaN  Southampton    no   True  
22  child       False  NaN   Queenstown   yes   True  
27    man        True    C  Southampton    no  False  
38  woman       False  NaN  Southampton    no  False  


In [25]:
mask2 = (titanic.age < 10) & (titanic.sex == 'female')
df_female_under10 = titanic.loc[mask2, :]
print(df_female_under10.head())

     survived  pclass     sex  age  sibsp  parch     fare embarked   class  \
10          1       3  female  4.0      1      1  16.7000        S   Third   
24          0       3  female  8.0      3      1  21.0750        S   Third   
43          1       2  female  3.0      1      2  41.5792        C  Second   
58          1       2  female  5.0      1      2  27.7500        S  Second   
119         0       3  female  2.0      4      2  31.2750        S   Third   

       who  adult_male deck  embark_town alive  alone  
10   child       False    G  Southampton   yes  False  
24   child       False  NaN  Southampton    no  False  
43   child       False  NaN    Cherbourg   yes  False  
58   child       False  NaN  Southampton   yes  False  
119  child       False  NaN  Southampton    no  False  


In [26]:
mask3 = (titanic.age < 10) | (titanic.age >= 60)
df_under10_morethan60 = titanic.loc[mask3, ['age', 'sex', 'alone']]
print(df_under10_morethan60.head())

     age     sex  alone
7    2.0    male  False
10   4.0  female  False
16   2.0    male  False
24   8.0  female  False
33  66.0    male   True


### 3-2. isin() 메소드 활용

isin() 메소드를 활용한 필터링: `DataFrame객체.isin(추출 값의 리스트)`

In [27]:
pd.set_option('display.max_columns', 10) # 출력할 열의 개수 한도

# 함께 탑승한 형제 또는 배우자의 수가 3, 4, 5인 승객만 따로 추출
mask3 = titanic['sibsp'] == 3
mask4 = titanic['sibsp'] == 4
mask5 = titanic['sibsp'] == 5

df_boolean = titanic[mask3 | mask4 | mask5]
print(df_boolean.head())

    survived  pclass     sex   age  sibsp  ...  adult_male  deck  embark_town  \
7          0       3    male   2.0      3  ...       False   NaN  Southampton   
16         0       3    male   2.0      4  ...       False   NaN   Queenstown   
24         0       3  female   8.0      3  ...       False   NaN  Southampton   
27         0       1    male  19.0      3  ...        True     C  Southampton   
50         0       3    male   7.0      4  ...       False   NaN  Southampton   

   alive  alone  
7     no  False  
16    no  False  
24    no  False  
27    no  False  
50    no  False  

[5 rows x 15 columns]


In [28]:
isin_filter = titanic['sibsp'].isin([3, 4, 5])
df_isin = titanic[isin_filter]
print(df_isin.head())

    survived  pclass     sex   age  sibsp  ...  adult_male  deck  embark_town  \
7          0       3    male   2.0      3  ...       False   NaN  Southampton   
16         0       3    male   2.0      4  ...       False   NaN   Queenstown   
24         0       3  female   8.0      3  ...       False   NaN  Southampton   
27         0       1    male  19.0      3  ...        True     C  Southampton   
50         0       3    male   7.0      4  ...       False   NaN  Southampton   

   alive  alone  
7     no  False  
16    no  False  
24    no  False  
27    no  False  
50    no  False  

[5 rows x 15 columns]


## 4. 데이터프레임 합치기

### 4-1. 데이터프레임 연결

기존 데이터프레임의 형태를 유지하면서 이어 붙이는 개념 -> concat() 함수  
: `pandas.concat(데이터프레임의 리스트, axis=0(default), join='outer'(default))`


In [29]:
import pandas as pd

df1 = pd.DataFrame({'a':['a0','a1','a2','a3'],
                   'b':['b0','b1','b2','b3'],
                   'c':['c0','c1','c2','c3']},
                  index=[0,1,2,3])

df2 = pd.DataFrame({'a':['a2','a3','a4','a5'],
                   'b':['b2','b3','b4','b5'],
                   'c':['c2','c3','c4','c5'],
                   'd':['d2','d3','d4','d5']},
                  index=[2,3,4,5])

print(df1, '\n')
print(df2)

    a   b   c
0  a0  b0  c0
1  a1  b1  c1
2  a2  b2  c2
3  a3  b3  c3 

    a   b   c   d
2  a2  b2  c2  d2
3  a3  b3  c3  d3
4  a4  b4  c4  d4
5  a5  b5  c5  d5


In [30]:
result1 = pd.concat([df1, df2]) # 위 아래 행 방향으로 이어 붙이듯 연결
print(result1)

    a   b   c    d
0  a0  b0  c0  NaN
1  a1  b1  c1  NaN
2  a2  b2  c2  NaN
3  a3  b3  c3  NaN
2  a2  b2  c2   d2
3  a3  b3  c3   d3
4  a4  b4  c4   d4
5  a5  b5  c5   d5


In [31]:
result2 = pd.concat([df1, df2], ignore_index=True) # 기존 행 인덱스를 무시하고 새로운 행 인덱스 설정(정수형 위치 인덱스)
print(result2)

    a   b   c    d
0  a0  b0  c0  NaN
1  a1  b1  c1  NaN
2  a2  b2  c2  NaN
3  a3  b3  c3  NaN
4  a2  b2  c2   d2
5  a3  b3  c3   d3
6  a4  b4  c4   d4
7  a5  b5  c5   d5


In [32]:
result3 = pd.concat([df1, df2], axis=1)
print(result3)

     a    b    c    a    b    c    d
0   a0   b0   c0  NaN  NaN  NaN  NaN
1   a1   b1   c1  NaN  NaN  NaN  NaN
2   a2   b2   c2   a2   b2   c2   d2
3   a3   b3   c3   a3   b3   c3   d3
4  NaN  NaN  NaN   a4   b4   c4   d4
5  NaN  NaN  NaN   a5   b5   c5   d5


In [33]:
result3_in = pd.concat([df1, df2], axis=1, join='inner')
print(result3_in)

    a   b   c   a   b   c   d
2  a2  b2  c2  a2  b2  c2  d2
3  a3  b3  c3  a3  b3  c3  d3


* **데이터 프레임과 시리즈를 좌우 열 방향으로 연결**  
데이터프레임에 열을 추가하는 것과 같고, 시리즈의 이름이 데이터프레임의 열 이름으로 변환된다.  
단, 데이터프레임의 행 인덱스와 시리즈의 인덱스가 같아야 한다.

In [34]:
sr1 = pd.Series(['e0','e1','e2','e3'], name='e')
sr2 = pd.Series(['f0','f1','f2'], name='f', index=[3,4,5])
sr3 = pd.Series(['g0','g1','g2','g3'], name='g')

In [35]:
# df1과 sr1을 좌우 열 방향으로 연결하기
result4 = pd.concat([df1, sr1], axis=1)
print(result4)

    a   b   c   e
0  a0  b0  c0  e0
1  a1  b1  c1  e1
2  a2  b2  c2  e2
3  a3  b3  c3  e3


In [36]:
# df2과 sr2을 좌우 열 방향으로 연결하기
result5 = pd.concat([df2, sr2], axis=1, sort=True) # 인덱스 중심 내림차순
print(result5)

    a   b   c   d    f
2  a2  b2  c2  d2  NaN
3  a3  b3  c3  d3   f0
4  a4  b4  c4  d4   f1
5  a5  b5  c5  d5   f2


In [37]:
# sr1과 sr3을 좌우 열 방향으로 연결하기
result6 = pd.concat([sr1, sr3], axis=1)
print(result6, "\n")

# sr1과 sr3을 위아래 행 방향으로 연결하기
result7 = pd.concat([sr1, sr3], axis=0)
print(result7)

    e   g
0  e0  g0
1  e1  g1
2  e2  g2
3  e3  g3 

0    e0
1    e1
2    e2
3    e3
0    g0
1    g1
2    g2
3    g3
dtype: object


### 4-2. 데이터프레임 병합

sql의 join 명령과 미슷한 방식으로 어떤 기준(key)에 의해 두 데이터프레임을 병합  
키가 되는 열이나 인덱스는 반드시 양쪽 데이터프레임에 모두 존재해야 한다.  
: `pandas.merge(df_left, df_right, how='inner', on=None)`  

`on=None` 옵션: 두 데이터프레임에 공통으로 속하는 모든 열을 기준으로 병합  
`how='inner'` 옵션: 기준이 되는 열의 데이터가 양쪽 데이터프레임에 공통으로 존재하는 교집합일 경우에만 추출

In [38]:
import pandas as pd

pd.set_option('display.max_columns', 10)                  # 출력할 최대 열의 개수
pd.set_option('display.max_colwidth', 20)                 # 출력할 열의 너비
pd.set_option('display.unicode.east_asian_width', True)  # 유니코드 사용 너비 조정

df1 = pd.read_excel('stock price.xlsx')
df2 = pd.read_excel('stock valuation.xlsx')

print(df1, '\n')
print(df2)

       id    stock_name          value   price
0  128940      한미약품   59385.666667  421000
1  130960        CJ E&M   58540.666667   98900
2  138250    엔에스쇼핑   14558.666667   13200
3  139480        이마트  239230.833333  254500
4  142280  녹십자엠에스     468.833333   10200
5  145990        삼양사   82750.000000   82000
6  185750        종근당   40293.666667  100500
7  192400    쿠쿠홀딩스  179204.666667  177500
8  199800          툴젠   -2514.333333  115400
9  204210  모두투어리츠    3093.333333    3475 

       id              name           eps     bps        per       pbr
0  130960            CJ E&M   6301.333333   54068  15.695091  1.829178
1  136480              하림    274.166667    3551  11.489362  0.887074
2  138040    메리츠금융지주   2122.333333   14894   6.313806  0.899691
3  139480            이마트  18268.166667  295780  13.931338  0.860437
4  145990            삼양사   5741.000000  108090  14.283226  0.758627
5  161390        한국타이어   5648.500000   51341   7.453306  0.820007
6  181710   NHN엔터테인먼트   2110.166667   784

In [39]:
merge_inner = pd.merge(df1, df2)
print(merge_inner)

       id    stock_name          value   price          name           eps  \
0  130960        CJ E&M   58540.666667   98900        CJ E&M   6301.333333   
1  139480        이마트  239230.833333  254500        이마트  18268.166667   
2  145990        삼양사   82750.000000   82000        삼양사   5741.000000   
3  185750        종근당   40293.666667  100500        종근당   3990.333333   
4  204210  모두투어리츠    3093.333333    3475  모두투어리츠     85.166667   

      bps        per       pbr  
0   54068  15.695091  1.829178  
1  295780  13.931338  0.860437  
2  108090  14.283226  0.758627  
3   40684  25.185866  2.470259  
4    5335  40.802348  0.651359  


In [40]:
merge_outer = pd.merge(df1, df2, how='outer', on='id')
print(merge_outer)

        id    stock_name          value     price              name  \
0   128940      한미약품   59385.666667  421000.0               NaN   
1   130960        CJ E&M   58540.666667   98900.0            CJ E&M   
2   138250    엔에스쇼핑   14558.666667   13200.0               NaN   
3   139480        이마트  239230.833333  254500.0            이마트   
4   142280  녹십자엠에스     468.833333   10200.0               NaN   
5   145990        삼양사   82750.000000   82000.0            삼양사   
6   185750        종근당   40293.666667  100500.0            종근당   
7   192400    쿠쿠홀딩스  179204.666667  177500.0               NaN   
8   199800          툴젠   -2514.333333  115400.0               NaN   
9   204210  모두투어리츠    3093.333333    3475.0      모두투어리츠   
10  136480           NaN            NaN       NaN              하림   
11  138040           NaN            NaN       NaN    메리츠금융지주   
12  161390           NaN            NaN       NaN        한국타이어   
13  181710           NaN            NaN       NaN   NHN엔터테인먼트   
14  207

In [41]:
# 왼쪽 데이터프레임 기준, 키 값 분리
merge_left = pd.merge(df1, df2, how='left', left_on='stock_name', right_on='name')
print(merge_left)

     id_x    stock_name          value   price      id_y          name  \
0  128940      한미약품   59385.666667  421000       NaN           NaN   
1  130960        CJ E&M   58540.666667   98900  130960.0        CJ E&M   
2  138250    엔에스쇼핑   14558.666667   13200       NaN           NaN   
3  139480        이마트  239230.833333  254500  139480.0        이마트   
4  142280  녹십자엠에스     468.833333   10200       NaN           NaN   
5  145990        삼양사   82750.000000   82000  145990.0        삼양사   
6  185750        종근당   40293.666667  100500  185750.0        종근당   
7  192400    쿠쿠홀딩스  179204.666667  177500       NaN           NaN   
8  199800          툴젠   -2514.333333  115400       NaN           NaN   
9  204210  모두투어리츠    3093.333333    3475  204210.0  모두투어리츠   

            eps       bps        per       pbr  
0           NaN       NaN        NaN       NaN  
1   6301.333333   54068.0  15.695091  1.829178  
2           NaN       NaN        NaN       NaN  
3  18268.166667  295780.0  13.931338  0.8

In [42]:
# 오른쪽 데이터프레임 기준, 키 값 분리
merge_right = pd.merge(df1, df2, how='right', left_on='stock_name', right_on='name')
print(merge_right)

       id_x    stock_name          value     price    id_y              name  \
0  130960.0        CJ E&M   58540.666667   98900.0  130960            CJ E&M   
1       NaN           NaN            NaN       NaN  136480              하림   
2       NaN           NaN            NaN       NaN  138040    메리츠금융지주   
3  139480.0        이마트  239230.833333  254500.0  139480            이마트   
4  145990.0        삼양사   82750.000000   82000.0  145990            삼양사   
5       NaN           NaN            NaN       NaN  161390        한국타이어   
6       NaN           NaN            NaN       NaN  181710   NHN엔터테인먼트   
7  185750.0        종근당   40293.666667  100500.0  185750            종근당   
8  204210.0  모두투어리츠    3093.333333    3475.0  204210      모두투어리츠   
9       NaN           NaN            NaN       NaN  207940  삼성바이오로직스   

            eps     bps        per       pbr  
0   6301.333333   54068  15.695091  1.829178  
1    274.166667    3551  11.489362  0.887074  
2   2122.333333   14894   6.313806  

In [43]:
# 불린 인덱싱과 결합하여 원하는 데이터 찾기
price = df1[df1['price'] < 50000]
print(price.head(), '\n')

value = pd.merge(price, df2) # 주가 50,000원 미만인 종목 중 df2에 밸류에이션 데이터를 가진 회사
print(value)

       id    stock_name         value  price
2  138250    엔에스쇼핑  14558.666667  13200
4  142280  녹십자엠에스    468.833333  10200
9  204210  모두투어리츠   3093.333333   3475 

       id    stock_name        value  price          name        eps   bps  \
0  204210  모두투어리츠  3093.333333   3475  모두투어리츠  85.166667  5335   

         per       pbr  
0  40.802348  0.651359  


### 4-3. 데이터프레임 결합

join() 메소드는 두 데이터프레임의 행 인덱스를 기준으로 결합하는 점에서 merge() 함수와 차이가 있다.  
(on=keys 옵션으로 다른 열을 기준으로 결합하는 것이 가능)

행 인덱스 기준으로 결합: `DataFrame1.join(DataFrame2, how='left')`

In [44]:
df1 = pd.read_excel('stock price.xlsx', index_col='id') # 행 인덱스를 id로 설정
df2 = pd.read_excel('stock valuation.xlsx', index_col='id')

df3 = df1.join(df2) # df1의 행 인덱스를 기준으로
print(df3)

          stock_name          value   price          name           eps  \
id                                                                        
128940      한미약품   59385.666667  421000           NaN           NaN   
130960        CJ E&M   58540.666667   98900        CJ E&M   6301.333333   
138250    엔에스쇼핑   14558.666667   13200           NaN           NaN   
139480        이마트  239230.833333  254500        이마트  18268.166667   
142280  녹십자엠에스     468.833333   10200           NaN           NaN   
145990        삼양사   82750.000000   82000        삼양사   5741.000000   
185750        종근당   40293.666667  100500        종근당   3990.333333   
192400    쿠쿠홀딩스  179204.666667  177500           NaN           NaN   
199800          툴젠   -2514.333333  115400           NaN           NaN   
204210  모두투어리츠    3093.333333    3475  모두투어리츠     85.166667   

             bps        per       pbr  
id                                     
128940       NaN        NaN       NaN  
130960   54068.0  15.695091  1.

In [45]:
df4 = df1.join(df2, how='inner')
print(df4)

          stock_name          value   price          name           eps  \
id                                                                        
130960        CJ E&M   58540.666667   98900        CJ E&M   6301.333333   
139480        이마트  239230.833333  254500        이마트  18268.166667   
145990        삼양사   82750.000000   82000        삼양사   5741.000000   
185750        종근당   40293.666667  100500        종근당   3990.333333   
204210  모두투어리츠    3093.333333    3475  모두투어리츠     85.166667   

           bps        per       pbr  
id                                   
130960   54068  15.695091  1.829178  
139480  295780  13.931338  0.860437  
145990  108090  14.283226  0.758627  
185750   40684  25.185866  2.470259  
204210    5335  40.802348  0.651359  


## 5. 그룹 연산

데이터를 집계, 변환, 필터링하는데 효율적이다.  
1. **분할(split)**: 데이터를 특정 조건에 의해 분할 - groupby() 메소드 사용
2. **적용(apply)**: 데이터를 집계, 변환, 필터링하는데 필요한 메소드 적용
3. **결합(combine)**: 2단계의 처리 결과를 하나로 결합

### 5-1. 그룹 객체 만들기(분할 단계)

* 1개 열을 기준으로 그룹화  

: `DataFrame객체.groupby(기준이 되는 열)`

In [46]:
import pandas as pd
import seaborn as sns

titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age','sex','class','fare','survived']]

print('승객 수:', len(df))
print(df.head()) 

승객 수: 891
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
1  38.0  female  First  71.2833         1
2  26.0  female  Third   7.9250         1
3  35.0  female  First  53.1000         1
4  35.0    male  Third   8.0500         0


In [47]:
# class열을 기준으로 분할
grouped = df.groupby(['class'])
print(grouped, '\n')

# 그룹 객체를 iteration으로 출력
for key, group in grouped:
    print('* key:', key)
    print('* number:', len(group))
    print(group.head())
    print('\n')


* key: First
* number: 216
     age     sex  class     fare  survived
1   38.0  female  First  71.2833         1
3   35.0  female  First  53.1000         1
6   54.0    male  First  51.8625         0
11  58.0  female  First  26.5500         1
23  28.0    male  First  35.5000         1


* key: Second
* number: 184
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
20  35.0    male  Second  26.0000         0
21  34.0    male  Second  13.0000         1


* key: Third
* number: 491
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0




In [48]:
# 연산 메소드 적용
average = grouped.mean()
print(average)

              age       fare  survived
class                                 
First   38.233441  84.154687  0.629630
Second  29.877630  20.662183  0.472826
Third   25.140620  13.675550  0.242363


In [49]:
# 개별 그룹 선택하기
group3 = grouped.get_group('Third')
print(group3.head())

    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0


* 여러 열을 기준으로 그룹화  

: `DataFrame객체.groupby(기준이 되는 열의 리스트)`

In [50]:
grouped_two = df.groupby(['class', 'sex'])

for key, group in grouped_two:
    print('* key:', key)
    print('* number:', len(group))
    print(group.head())
    print('\n')

* key: ('First', 'female')
* number: 94
     age     sex  class      fare  survived
1   38.0  female  First   71.2833         1
3   35.0  female  First   53.1000         1
11  58.0  female  First   26.5500         1
31   NaN  female  First  146.5208         1
52  49.0  female  First   76.7292         1


* key: ('First', 'male')
* number: 122
     age   sex  class      fare  survived
6   54.0  male  First   51.8625         0
23  28.0  male  First   35.5000         1
27  19.0  male  First  263.0000         0
30  40.0  male  First   27.7208         0
34  28.0  male  First   82.1708         0


* key: ('Second', 'female')
* number: 76
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
41  27.0  female  Second  21.0000         0
43   3.0  female  Second  41.5792         1
53  29.0  female  Second  26.0000         1


* key: ('Second', 'male')
* number: 108
     age   sex   class  fare  survived
17   NaN  male 

In [51]:
average_two = grouped_two.mean()
print(average_two)

# => 여성의 생존률이 남성에 비해 월등히 높다. 3등석 여성 승객의 생존율은 1,2등석에 비해 낮다.

                     age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447


In [52]:
group3f = grouped_two.get_group(('Third', 'female')) # 멀티인덱스
print(group3f.head())

     age     sex  class     fare  survived
2   26.0  female  Third   7.9250         1
8   27.0  female  Third  11.1333         1
10   4.0  female  Third  16.7000         1
14  14.0  female  Third   7.8542         0
18  31.0  female  Third  18.0000         0


### 5-2. 그룹 연산 메소드(적용 - 결합 단계)

* 데이터 집계  
: 그룹 객체에 다양한 연산 적용

In [55]:
grouped = df.groupby(['class'])

std_all = grouped.std()
print(std_all, '\n')
print(type(std_all))

              age       fare  survived
class                                 
First   14.802856  78.380373  0.484026
Second  14.001077  13.417399  0.500623
Third   12.495398  11.778142  0.428949 

<class 'pandas.core.frame.DataFrame'>


In [62]:
std_fare = grouped.fare.std()
print(std_fare, '\n')
print(type(std_fare))

class
First     78.380373
Second    13.417399
Third     11.778142
Name: fare, dtype: float64 

<class 'pandas.core.series.Series'>


집계 연산을 처리하는 사용자 정의 함수를 그룹 객체에 적용할 때  
: agg() 메소드 적용  
: `group객체.agg(매핑 함수)`

In [63]:
def min_max(x):
    return x.max() - x.min()

agg_minmax = grouped.agg(min_max)
print(agg_minmax.head())

          age      fare  survived
class                            
First   79.08  512.3292         1
Second  69.33   73.5000         1
Third   73.58   69.5500         1


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


* 모든 열에 여러 함수를 매핑: `group객체.agg([함수1, 함수2, 함수3,...])`  
* 각 열마다 다른 함수를 매핑: `group객체.agg({'열1':함수1, '열2':함수2, ...})`

In [64]:
# 여러 함수를 각 열에 동일하게 적용하여 집계
agg_all = grouped.agg(['min', 'max'])
print(agg_all.head(), '\n')

# 각 열마다 다른 함수를 적용하여 집계 
agg_sep = grouped.agg({'fare':['min','max'], 'age':'mean'})
print(agg_sep.head())

         age           sex       fare           survived    
         min   max     min   max  min       max      min max
class                                                       
First   0.92  80.0  female  male  0.0  512.3292        0   1
Second  0.67  70.0  female  male  0.0   73.5000        0   1
Third   0.42  74.0  female  male  0.0   69.5500        0   1 

       fare                  age
        min       max       mean
class                           
First   0.0  512.3292  38.233441
Second  0.0   73.5000  29.877630
Third   0.0   69.5500  25.140620


* 그룹 연산 데이터 변환  

transform() 메소드:  
그룹별 집계 대신 각 원소의 본래 행 인덱스와 열 이름을 기준으로 연산 결과를 반환한다.  
(그룹 연산의 결과를 원본 데이터프레임과 같은 형태로 변형해 정리)  

: `group객체.transform(매핑 함수)`

In [68]:
# 그룹별 age 열의 평균 집계 연산
age_mean = grouped.age.mean()
print(age_mean, '\n')

# 그룹별 age 열의 표준편차 집계 연산
age_std = grouped.age.std()
print(age_std, '\n')
print('\n')

# 그룹 객체의 age 열을 iteration으로 z-score를 계산하여 출력
for key, group in grouped.age:
    group_zscore = (group - age_mean.loc[key])/age_std.loc[key]
    print('* origin :', key)
    print(group_zscore.head(3))
    print('\n')

class
First     38.233441
Second    29.877630
Third     25.140620
Name: age, dtype: float64 

class
First     14.802856
Second    14.001077
Third     12.495398
Name: age, dtype: float64 



* origin : First
1   -0.015770
3   -0.218434
6    1.065103
Name: age, dtype: float64


* origin : Second
9    -1.134029
15    1.794317
17         NaN
Name: age, dtype: float64


* origin : Third
0   -0.251342
2    0.068776
4    0.789041
Name: age, dtype: float64




In [69]:
# transform() 메소드를 이용하여 age 열의 데이터를 z-score로 변환
def z_score(x):
    return (x - x.mean()) / x.std()

age_zscore = grouped.age.transform(z_score)
print(age_zscore.loc[[1,9,0]], '\n')  # 1, 2, 3 그룹의 첫 데이터 확인
print(len(age_zscore), '\n')
print(age_zscore.loc[0:9], '\n')
print(type(age_zscore))

1   -0.015770
9   -1.134029
0   -0.251342
Name: age, dtype: float64 

891 

0   -0.251342
1   -0.015770
2    0.068776
3   -0.218434
4    0.789041
5         NaN
6    1.065103
7   -1.851931
8    0.148805
9   -1.134029
Name: age, dtype: float64 

<class 'pandas.core.series.Series'>


* 그룹 객체 필터링  

: `group객체.filter(조건식 함수)`

In [70]:
# 데이터 개수가 200개 이상인 그룹만을 필터링하여 데이터프레임으로 반환 -> 'First', 'Third'
grouped_filter = grouped.filter(lambda x: len(x) >= 200)
print(grouped_filter.head(), '\n')
print(type(grouped_filter))

    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
1  38.0  female  First  71.2833         1
2  26.0  female  Third   7.9250         1
3  35.0  female  First  53.1000         1
4  35.0    male  Third   8.0500         0 

<class 'pandas.core.frame.DataFrame'>


In [71]:
# age 열의 평균이 30보다 작은 그룹만을 필터링하여 데이터프레임으로 반환 -> 'Second', 'Third'
age_filter = grouped.filter(lambda x: x.age.mean() < 30)
print(age_filter.tail(), '\n')
print(type(age_filter))

      age     sex   class    fare  survived
884  25.0    male   Third   7.050         0
885  39.0  female   Third  29.125         0
886  27.0    male  Second  13.000         0
888   NaN  female   Third  23.450         0
890  32.0    male   Third   7.750         0 

<class 'pandas.core.frame.DataFrame'>


* 그룹 객체에 함수 매핑  

범용 메소드: `group객체.apply(매핑 함수)`


In [76]:
agg_grouped = grouped.apply(lambda x: x.describe())
print(agg_grouped)

                     age        fare    survived
class                                           
First  count  186.000000  216.000000  216.000000
       mean    38.233441   84.154687    0.629630
       std     14.802856   78.380373    0.484026
       min      0.920000    0.000000    0.000000
       25%     27.000000   30.923950    0.000000
       50%     37.000000   60.287500    1.000000
       75%     49.000000   93.500000    1.000000
       max     80.000000  512.329200    1.000000
Second count  173.000000  184.000000  184.000000
       mean    29.877630   20.662183    0.472826
       std     14.001077   13.417399    0.500623
       min      0.670000    0.000000    0.000000
       25%     23.000000   13.000000    0.000000
       50%     29.000000   14.250000    0.000000
       75%     36.000000   26.000000    1.000000
       max     70.000000   73.500000    1.000000
Third  count  355.000000  491.000000  491.000000
       mean    25.140620   13.675550    0.242363
       std     12.49

In [77]:
def z_score(x):
    return (x - x.mean()) / x.std()

age_zscore = grouped.age.apply(z_score)
print(age_zscore.head())

0   -0.251342
1   -0.015770
2    0.068776
3   -0.218434
4    0.789041
Name: age, dtype: float64


In [79]:
#  필터링: age열의 데이터 평균이 30보다 작은 그룹만을 필터링하여 출력
age_filter = grouped.apply(lambda x: x.age.mean() < 30)
print(age_filter, '\n')

for x in age_filter.index:
    if age_filter[x]==True:
        age_filter_df = grouped.get_group(x)
        print(age_filter_df.head())
        print('\n')

class
First     False
Second     True
Third      True
dtype: bool 

     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
20  35.0    male  Second  26.0000         0
21  34.0    male  Second  13.0000         1


    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0




## 6. 멀티 인덱스

판다스는 행 인덱스를 여러 레벨로 구현할 수 있도록 멀티 인덱스(MultiIndex) 클래스를 지원한다.

In [80]:
# class 열, sex 열을 기준으로 분할
grouped = df.groupby(['class', 'sex'])

# 그룹 객체에 연산 메소드 적용
gdf = grouped.mean() # 멀티 인덱스를 가진 데이터 프레임
print(gdf, '\n')
print(type(gdf))

                     age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447 

<class 'pandas.core.frame.DataFrame'>


In [85]:
# 멀티 인덱스에서 하나의 인덱스만 사용하는 방법
print(gdf.loc['First'], '\n')
print(gdf.xs('First', level='class'), '\n')

# 멀티 인덱스에서 두 개의 인덱스를 사용하는 방법
print(gdf.loc[('First', 'female')], '\n')

# sex 값이 male인 행을 선택하여 출력
print(gdf.xs('male', level='sex'))

              age        fare  survived
sex                                    
female  34.611765  106.125798  0.968085
male    41.281386   67.226127  0.368852 

              age        fare  survived
sex                                    
female  34.611765  106.125798  0.968085
male    41.281386   67.226127  0.368852 

age          34.611765
fare        106.125798
survived      0.968085
Name: (First, female), dtype: float64 

              age       fare  survived
class                                 
First   41.281386  67.226127  0.368852
Second  30.740707  19.741782  0.157407
Third   26.507589  12.661633  0.135447


## 7. 피벗

pivot_table() 함수 적용  
피벗 테이블을 구성하는 4가지 요소(행 인덱스, 열 인덱스, 데이터 값, 데이터 집계 함수)에 적용할 데이터프레임의 열을 각각 지정하여 함수의 인자로 전달

In [86]:
import pandas as pd
import seaborn as sns

pd.set_option('display.max_columns', 10) 
pd.set_option('display.max_colwidth', 20) 

titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age','sex','class','fare','survived']]
print(df.head(), '\n')

# 행, 열, 값, 집계에 사용할 열을 1개씩 지정 - 평균 집계
pdf1 = pd.pivot_table(df,
                     index='class',  # 행 위치에 들어갈 열
                     columns='sex',  # 열 위치에 들어갈 열
                     values='age',   # 데이터로 사용할 열
                     aggfunc='mean') # 데이터 집계 함수

print(pdf1.head())

    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
1  38.0  female  First  71.2833         1
2  26.0  female  Third   7.9250         1
3  35.0  female  First  53.1000         1
4  35.0    male  Third   8.0500         0 

sex        female       male
class                       
First   34.611765  41.281386
Second  28.722973  30.740707
Third   21.750000  26.507589


In [87]:
# 값에 적용하는 집계 함수 2개 이상 지정 가능 - 생존율, 생존자 수 집계
pdf2 = pd.pivot_table(df,
                     index='class',
                     columns='sex',
                     values='survived',
                     aggfunc=['mean', 'sum'])

print(pdf2.head()) # 열 구조는 2중 멀티 인덱스

            mean              sum     
sex       female      male female male
class                                 
First   0.968085  0.368852     91   45
Second  0.921053  0.157407     70   17
Third   0.500000  0.135447     72   47


In [90]:
# 행, 열, 값에 사용할 열을 2개 이상 지정 가능 - 평균 나이, 최대 요금 집계
pdf3 = pd.pivot_table(df,
                     index=['class','sex'],
                     columns='survived',
                     values=['age','fare'],
                     aggfunc=['mean', 'max'])

pd.set_option('display.max_columns', 10)
print(pdf3.head(), '\n')

# 행, 열 구조
print(pdf3.index)
print(pdf3.columns)

                    mean                                      max        \
                     age                   fare               age         
survived               0          1           0           1     0     1   
class  sex                                                                
First  female  25.666667  34.939024  110.604167  105.978159  50.0  63.0   
       male    44.581967  36.248000   62.894910   74.637320  71.0  80.0   
Second female  36.000000  28.080882   18.250000   22.288989  57.0  55.0   
       male    33.369048  16.022000   19.488965   21.095100  70.0  62.0   
Third  female  23.818182  19.329787   19.773093   12.464526  48.0  63.0   

                                 
                 fare            
survived            0         1  
class  sex                       
First  female  151.55  512.3292  
       male    263.00  512.3292  
Second female   26.00   65.0000  
       male     73.50   39.0000  
Third  female   69.55   31.3875   

MultiIndex([( 'F

In [95]:
# xs 인덱서 사용 - 행 선택(default: axis=0)
print(pdf3.xs('First'))
print('\n')
print(pdf3.xs(('First', 'female')))
print('\n')
print(pdf3.xs('male', level='sex'))
print('\n')
print(pdf3.xs(('Second','male'), level=[0,'sex'])) # 'sex' 대신 숫자형 레벨 1도 가능

               mean                                      max                \
                age                   fare               age          fare   
survived          0          1           0           1     0     1       0   
sex                                                                          
female    25.666667  34.939024  110.604167  105.978159  50.0  63.0  151.55   
male      44.581967  36.248000   62.894910   74.637320  71.0  80.0  263.00   

                    
                    
survived         1  
sex                 
female    512.3292  
male      512.3292  


            survived
mean  age   0            25.666667
            1            34.939024
      fare  0           110.604167
            1           105.978159
max   age   0            50.000000
            1            63.000000
      fare  0           151.550000
            1           512.329200
Name: (First, female), dtype: float64


               mean                                    max    

In [99]:
# xs 인덱서 사용 - 열 선택(axis=1 설정)
print(pdf3.xs('mean', axis=1))
print('\n')
print(pdf3.xs(('mean','age'), axis=1))
print('\n')
print(pdf3.xs(1, level='survived', axis=1)) # survived 레벨이 1인 데이터 선택
print('\n')
print(pdf3.xs(('max','fare',0), level=[0,1,2], axis=1)) # max, fare, survived=0 인 데이터 선택 => 구조받지 못한 승객들 객실 요금의 최대값

                     age                   fare            
survived               0          1           0           1
class  sex                                                 
First  female  25.666667  34.939024  110.604167  105.978159
       male    44.581967  36.248000   62.894910   74.637320
Second female  36.000000  28.080882   18.250000   22.288989
       male    33.369048  16.022000   19.488965   21.095100
Third  female  23.818182  19.329787   19.773093   12.464526
       male    27.255814  22.274211   12.204469   15.579696


survived               0          1
class  sex                         
First  female  25.666667  34.939024
       male    44.581967  36.248000
Second female  36.000000  28.080882
       male    33.369048  16.022000
Third  female  23.818182  19.329787
       male    27.255814  22.274211


                    mean               max          
                     age        fare   age      fare
class  sex                                          
First  fe